# Summary table for **COMPAS** data

Here we extract the summary measures for *COMPAS* data for four classifiers: (1) baseline, (2) project, (3) reduction, and (4) SenSR.
For each of the classifiers we perform gradient flow attack with step size = 5e-3 where $i$ is the current step number. We perform 200 steps of attacks on each data points. 

First, we load the summary data for all experiments, and compile it in a panda dataframe. 

In [20]:
import re
import pandas as pd
import json
with open('summary/summary_gfa1.txt', 'r') as f:
    d = json.load(f)


In [21]:

data = pd.DataFrame(d)
#data['reject'] = data['pval'] < 0.05
lr = 2e-3
data_lr = data.loc[data['lr'] == lr]
data_lr = data_lr.rename(columns= {'average_odds_difference_gen': 'AOD-gen', 'average_odds_difference_race': 'AOD-race', 'statistical_parity_difference_gen': 'SPD-gen', 'statistical_parity_difference_race': 'SPD-race',\
                                   'equal_opportunity_difference_race': 'EOD-race', 'equal_opportunity_difference_gen': 'EOD-gen'})
data_lr.dtypes

algo              object
seed              object
lr               float64
accuracy         float64
lb               float64
pval             float64
bal_acc          float64
gap_rms_gen      float64
mean_gap_gen     float64
max_gap_gen      float64
AOD-gen          float64
EOD-gen          float64
SPD-gen          float64
gap_rms_race     float64
mean_gap_race    float64
max_gap_race     float64
AOD-race         float64
EOD-race         float64
SPD-race         float64
lb-t2            float64
dtype: object

data_lr

We only extract the following measures: balanced accuracy, average odds difference for gender and race and lower bound and proportion of times the test being rejected. 

In [22]:
measure = ['bal_acc', 'AOD-gen', 'AOD-race', 'lb', 'lb-t2']#, 'reject']
agg_dict = dict()
for key in measure:
    agg_dict[key] = ['mean', 'std']
result = data_lr.groupby(['algo'], as_index=False).agg(agg_dict)
result

algo   bal_acc             AOD-gen            AOD-race            \
                  mean       std      mean       std      mean       std   
0   baseline  0.658564  0.011056  0.201118  0.042164  0.271933  0.035886   
1    project  0.646496  0.011121  0.086918  0.042230  0.191357  0.029364   
2  reduction  0.655648  0.012991  0.011637  0.058538  0.072250  0.048846   
3      sensr  0.651110  0.008678  0.106435  0.033477  0.188971  0.031524   

         lb               lb-t2            
       mean       std      mean       std  
0  1.339781  0.030120  1.436864  0.068734  
1  1.311500  0.540452  1.206805  0.304710  
2  1.366373  0.028704  1.570820  0.072378  
3  1.183882  0.307340  1.169214  0.224899

In [4]:
rows = ['sensr', 'reduction', 'baseline', 'project']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
colnames=['Balanced Acc', '$\\text{AOD}_{\\text{gen}}$', '$\\text{AOD}_{\\text{race}}$', '$T_n$', 'Rejection Prop']


In [5]:
for i, c in enumerate(measure):
    if c == 'reject':
        idx_best = (1-result[c]['mean']).idxmin()
        result[colnames[i]] = (result[c]['mean']).apply("{:.1f}".format)
        #result.at[idx_best, colnames[i]] = '\\textbf{' + result[colnames[i]][idx_best] + '}'
        
    elif c == 'bal_acc':
        idx_best = (result[c]['mean']).idxmax()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    elif c == 'lb':
        idx_best = (result[c]['mean']).idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    else:
        idx_best = (result[c]['mean']).abs().idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val


In [6]:
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
    res = result[colnames].rename(index=ind)
res

,Balanced Acc,$\text{AOD}_{\text{gen}}$,$\text{AOD}_{\text{race}}$,$T_n$,Rejection Prop
,,,,,
Baseline,0.675$\pm$0.013,0.218$\pm$0.041,0.260$\pm$0.026,2.385$\pm$0.262,1.0
Project,0.641$\pm$0.017,0.039$\pm$0.029,0.227$\pm$0.021,1.161$\pm$0.145,0.2
Reduction,0.652$\pm$0.012,-0.014$\pm$0.054,0.037$\pm$0.039,1.763$\pm$0.069,1.0
SenSR,0.640$\pm$0.022,0.046$\pm$0.031,0.237$\pm$0.018,1.098$\pm$0.061,0.0


In [7]:
print(res.to_latex(escape=False, column_format='l' + 'c'*len(colnames)))

\begin{tabular}{lccccc}
\toprule
{} &     Balanced Acc & $\text{AOD}_{\text{gen}}$ & $\text{AOD}_{\text{race}}$ &            $T_n$ & Rejection Prop \\
{} \\
\midrule
Baseline  &  0.675$\pm$0.013 &           0.218$\pm$0.041 &            0.260$\pm$0.026 &  2.385$\pm$0.262 &            1.0 \\
Project   &  0.641$\pm$0.017 &           0.039$\pm$0.029 &            0.227$\pm$0.021 &  1.161$\pm$0.145 &            0.2 \\
Reduction &  0.652$\pm$0.012 &          -0.014$\pm$0.054 &            0.037$\pm$0.039 &  1.763$\pm$0.069 &            1.0 \\
SenSR     &  0.640$\pm$0.022 &           0.046$\pm$0.031 &            0.237$\pm$0.018 &  1.098$\pm$0.061 &            0.0 \\
\bottomrule
\end{tabular}



In [24]:
import re
import pandas as pd
import json
with open('summary/summary_faith1.txt', 'r') as f:
    d = json.load(f)

In [26]:
data = pd.DataFrame(d)
data = data.rename(columns= {'exp': 'algo'})
data.dtypes

psi          float64
test-stat    float64
c-upper      float64
algo          object
iter           int64
dtype: object

In [28]:
measure = ['psi']#, 'reject']
agg_dict = dict()
for key in measure:
    agg_dict[key] = ['mean', 'std']
result2 = data.groupby(['algo'], as_index=False).agg(agg_dict)
result2

algo       psi          
                  mean       std
0   baseline  0.274339  0.158281
1    project  0.110635  0.155974
2  reduction  0.207099  0.117275
3      sensr  0.114069  0.145619

In [ ]:
data